# **GenoVarDis@IberLEF2024: Automatic Genomic Variants and Related Diseases using Named Entity Recognition with Large Language Models**

## Autor: Víctor Manuel Oliveros Villena


In [ ]:
# Esta es la ruta a la raíz de nuestro Drive.
# Si se prefiere otra ruta, añadir los directorios pertinentes a continuación de esta.
path_drive = '/content/drive/MyDrive'

### **Configuración del entorno**

In [ ]:
# Instalacion de bibliotecas necesarias
!pip install -q openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
import openai
import re

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


### **Lectura de archivos**

In [ ]:
# Ruta al archivo TSV en Google Drive
path_test_text = path_drive + '/GenoVarDis/Data/test_text.tsv'

# Lectura del archivo TSV utilizando pandas
data = pd.read_csv(path_test_text, sep='\t')

### **Configuración del modelo GPT**

In [ ]:
# Por motivos de privacidad, las URL de punto de acceso y la API Key no han
# podido ser compartidas
openai.api_type = 'azure'         # Tipo de API Azure
openai.api_base = 'URL'           # URL del punto de acceso
openai.api_version = '2023-05-15' # Versión de la API a emplear
openai.api_key = 'KEY'            # Clave de la API para autenticación

In [ ]:
# Prompt a emplear en el modelo GPT para servirle como contexto de la tarea a
# realizar
system_content = '''Eres un modelo de inteligencia artificial especializado en el reconocimiento de entidades nombradas (NER) en el campo de la biomedicina.
Debes identificar sola y únicamente los siguientes tipos de entidades en el texto: "Gene", "Disease", "DNAMutation", "SNP", "DNAAllele", "NucleotideChange-BaseChange", "OtherMutation", "Transcript". Cualquier otra etiqueta daría lugar a error.
Recuerda que no todas estas entidades necesariamente aparecerán en cada caso clínico, pero debes ser riguroso al clasificarlas según corresponda.
Los resultados, siempre y cuando se hayan detectado entidades, deben mostrarse exactamente en el formato: {tipo entidad} --> {entidad}\n. Escribe cada entidad encontrada en una línea distinta, aunque sean del mismo tipo.
Además, es muy importante que aparezcan por orden de aparición y escritas exactamente igual que en el texto (tildes, mayúsculas). En caso de no encontrar entidades devuelve un string vacío: '' '''

### **Ejecución del modelo y generación del archivo TSV con los resultados**

In [ ]:
# Ruta donde guardaremos el archivo con las anotaciones realizadas
filename = path_drive + '/GenoVarDis/EvaluationTSV/annotationGPT3.5Turbo.tsv'
# Nombre de las columnas de dicho archivo TSV
tags = 'pmid\tfilename\tlabel\toffset1\toffset2\tspan\n'
# Tipos de entidades
labels = ["Gene", "Disease", "DNAMutation", "SNP", "DNAAllele", "NucleotideChange-BaseChange", "OtherMutation", "Transcript"]

In [ ]:
def detect_entities(text):
    """
    Detecta entidades en el texto proporcionado utilizando el modelo GPT-3.5 Turbo de OpenAI.

    Args:
        text (str): Caso clinico donde identificar las entidades.

    Returns:
        str: Un string que contiene las entidades detectadas en el formato '{tipo entidad} --> {entidad}\n'.
    """
    response = openai.ChatCompletion.create(
        engine='gpt-35-turbo-model',
        messages=[
            {'role': 'system', 'content': system_content}, # Nuestro prompt
            {'role': 'user', 'content': text}              # Caso clinico
        ]
    )
    return response['choices'][0]['message']['content']

with open(filename, 'w') as file:
  file.write(tags)  # Escribe las etiquetas de las columnas en el TSV
  # Para cada caso clinico...
  for i, clinic_case in data.iterrows():
    pmid = clinic_case['pmid']  # Obtenemos pmid
    ann = clinic_case['filename'].replace('.txt', '.ann') # Generamos nombre del archivo
    text = clinic_case['text']  # Obtenemos el texto del caso clinico
    # Establece los offsets iniciales
    offset1 = 0
    offset2 = 0
    set_of1 = set()
    set_of2 = set()

    # Detecta las entidades mediante GPT
    entities = detect_entities(text)
    # Guarda una lista con cada una de las entidades
    list_entitites = entities.split('\n')

    # Mostramos los resultados
    print(list_entitites)

    # Por cada entidad detectada en un caso clinico...
    for entity in list_entitites:
      # Dividimos la entidad de su tipo
      entity_split = entity.split('-->')
      # Si el formato devuelto por GPT es incorrecto, ignoramos dicha entidad
      if entity == '' or len(list_entitites) == 1 or len(entity_split) != 2:
        continue
      # Ajustamos el formato de las entidades y sus tipos
      label, span = entity_split
      span = span.strip()
      label = label.strip()
      # Si el tipo de entidad no corresponde con los indicados, ignoramos la entidad
      if label not in labels:
        continue
      # Patron de busqueda de la entidad
      pattern = rf'\b{re.escape(span)}\b'
      # Iterador con todas las coincidencias de esta en el caso clinico
      matches = re.finditer(pattern, text)

      # Guarda la posicion de inicio y fin de la entidad en el caso clinico
      for m in matches:
        if m.start() not in set_of1 and m.end() not in set_of2:
          offset1 = m.start()
          offset2 = m.end()
          set_of1.add(offset1)
          set_of2.add(offset2)
          break
      # Añade la entidad al archivo TSV
      file.write(f"{pmid}\t{ann}\t{label}\t{offset1}\t{offset2}\t{span}\n")


['Gene --> PTCH1', 'DNAMutation --> c.930delC', 'Disease --> Síndrome de Gorlin', 'OtherMutation --> heterocigosis']
['Disease --> sindrome cri du chat (SCDC)', 'DNAMutation --> delecion', 'OtherMutation --> traslocacion', 'OtherMutation --> inversion', 'Gene --> TERT', 'NucleotideChange-BaseChange --> 5p15.33 (151537-3057771)x1']
['Gene --> PMP22', 'Disease --> Charcot-Marie-Tooth', 'Disease --> CMT', 'Disease --> CMT1A', 'DNAMutation --> duplicacion', 'NucleotideChange-BaseChange --> 1,5 Mb', 'OtherMutation --> mutacion', 'Transcript --> tecnica de reaccion en cadena de la polimerasa', 'Transcript --> tecnica de hibridacion in situ fluorescente']
['Factores neurotróficos --> neurotroficos', 'Trastorno por déficit de atención/hiperactividad --> TDAH', 'Neurotrofinas --> neurotrofinas', 'Proteínas --> proteinas', 'Sistema nervioso central --> sistema nervioso central', 'Sistema nervioso periférico --> sistema nervioso periferico', 'Conexiones sinápticas --> conexiones sinapticas', 'Fac